In [ ]:
from pathlib import Path
import pickle
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
from skopt import BayesSearchCV
from joblib import parallel_backend

from duqling_interface import DuqlingInterface
from model_search_spaces import get_models
from plot_performance import plot_bayes_cv_rmse, heatmap

In [ ]:
duq = DuqlingInterface()

univariate_funcs = duq.list_functions(response_type='uni').fname

In [ ]:
def record_performance(bayes_search, X_test:np.array, y_test:np.array):
    """
    Compute the test RMSE, standard deviation (sigma), and Pearson's R value
    and record them in the cross validation results dictionary.
    """
    y_pred = bayes_search.best_estimator_.predict(X_test)
    rmse  = np.sqrt(mean_squared_error(y_test, y_pred))
    sigma = y_test.std(ddof=0)
    r     = np.corrcoef(y_pred, y_test)[0, 1]
    bayes_search.cv_results_['test_rmse'] = rmse
    bayes_search.cv_results_['sigma']     = sigma
    bayes_search.cv_results_['r_val']     = r

In [ ]:
def save_cv_results(cv_results:dict, savepath:Path):
    savepath.parent.mkdir(parents=True, exist_ok=True)
    with open(savepath, 'wb') as f:
        pickle.dump(cv_results, f)

In [ ]:
for duqling_func_name in univariate_funcs[2:3]:

    X_tr, y_tr = duq.generate_data(duqling_func_name, n_samples=1000, seed=42)
    X_va, y_va = duq.generate_data(duqling_func_name, n_samples=1000, seed=43)
    X_te, y_te = duq.generate_data(duqling_func_name, n_samples=1000, seed=44)

    X_tune = np.vstack([X_tr, X_va])
    y_tune = np.hstack([y_tr, y_va])

    test_fold = np.r_[np.full(len(X_tr), -1), np.zeros(len(X_va))]
    ps = PredefinedSplit(test_fold)

    n_features = X_tr.shape[1]
    models = get_models(n_features)
    models.pop('gpr')
    for model_name in models:
        total_iters = models[model_name]['n_iter']
        bar = tqdm(total=total_iters, desc=f"{model_name.upper()} on {duqling_func_name}", unit="iter")
        def update_tqdm_bar(_): bar.update()

        bayes_search = BayesSearchCV(
            **models[model_name],
            cv=ps,
            scoring='neg_mean_squared_error',
            random_state=42,
            n_jobs=-1
        )
        with parallel_backend('threading', n_jobs=-1):
            bayes_search.fit(X_tune, y_tune, callback=update_tqdm_bar)
        bar.close()
        
        record_performance(bayes_search, X_te, y_te)

        # plot_bayes_cv_rmse(bayes_search.cv_results_, model_name.upper(), duqling_func_name)

        savepath = Path("models", model_name, duqling_func_name, f"cv_no_fold_results.pkl")
        save_cv_results(bayes_search.cv_results_, savepath)

In [ ]:
import xarray as xr

metrics     = ["test_rmse", "sigma", "r_val"]
model_names = [m for m in get_models(1).keys() if m != 'gpr']

arr = np.full((len(metrics), len(model_names), len(univariate_funcs)), np.nan)

for j, model in enumerate(model_names):
    for k, func in enumerate(univariate_funcs):
        pkl = Path('models', model, func, 'cv_no_fold_results.pkl')
        if not pkl.exists():
            continue
        with pkl.open("rb") as fh:
            data = pickle.load(fh)
        for i, m in enumerate(metrics):
            arr[i, j, k] = data[m]

summary = xr.DataArray(
    arr,
    coords={"metric": metrics, "model": model_names, "function": univariate_funcs},
    dims=["metric", "model", "function"]
)

def metric_df(metric: str) -> pd.DataFrame:
    """Return a model and function DataFrame for a single metric."""
    return summary.sel(metric=metric).to_pandas()

df_rmse = metric_df('test_rmse')
df_r    = metric_df('r_val')
df_std  = metric_df('sigma')

cols_to_drop = [
    # 'circuit', 'cantilever_S', 'banana', 'cube3_rotate', 'steel_column',
    'const_fn', 'const_fn3', 'const_fn15', 'cube3_rotate'
]
fig1 = heatmap(df_rmse.drop(cols_to_drop, axis=1), "Test RMSE")
fig2 = heatmap((df_rmse/df_std).drop(cols_to_drop, axis=1), "Test RMSE / \u03C3")
fig3 = heatmap((df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1), "(Test RMSE / \u03C3) > 1")

df_filtered = (df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1)
drop_cols = df_filtered.columns[df_filtered.isnull().all()]
fig4 = heatmap(df_filtered.drop(drop_cols, axis=1), "(Test RMSE / \u03C3) > 1")

fig1.show()
fig2.show()
fig3.show()
fig4.show()

In [ ]:
heatmap(df_rmse[df_filtered.drop(drop_cols, axis=1).columns], 'Test RMSE at Outliers')